In [1]:
#imports
import os
import pandas as pd
from env import username, host, password 
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from wrangle import get_zillow_data, prepare_zillow, train_validate_test_split, train_validate_test, impute_nulls, split_X_y

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
import statsmodels.api as sm

In [2]:
#get our data, turn into .csv, and check shape
df = get_zillow_data()
df.shape

(77575, 68)

In [3]:
#prepare and clean our data for use
df = prepare_zillow(df)

In [4]:
df['bathsandbeds'] = df.baths + df.beds

In [5]:
df.isna().sum()

propertylandusetypeid           0
parcel_id                       0
id                              0
baths                           0
beds                            0
calculatedbathnbr              14
sqft                            0
finishedsquarefeet12          181
fullbathcnt                    14
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcity                  918
regionidzip                     0
roomcnt                         0
year_built                      0
structuretaxvaluedollarcnt     56
tax_value                       0
assessmentyear                  0
landtaxvaluedollarcnt           0
tax_amount                      0
censustractandblock            46
logerror                        0
transactiondate                 0
propertylandusedesc             0
LA                              0
Orange                          0
Ventura       

In [6]:
#split into train, validate, test
train, validate, test = train_validate_test_split(df, 'logerror', seed=1349)

In [7]:
train, validate, test = impute_nulls(train, validate, test, strategy='most_frequent', col_list=['calculatedbathnbr', 'fullbathcnt', 'regionidcity', 'regionidzip', 'censustractandblock'])

In [8]:
train, validate, test = impute_nulls(train, validate, test, strategy='median', col_list=['finishedsquarefeet12', 'lotsizesquarefeet', 'structuretaxvaluedollarcnt', 'tax_value', 'landtaxvaluedollarcnt', 'tax_amount'])

In [9]:
train.isna().sum()

propertylandusetypeid          0
parcel_id                      0
id                             0
baths                          0
beds                           0
calculatedbathnbr              0
sqft                           0
finishedsquarefeet12           0
fullbathcnt                    0
latitude                       0
longitude                      0
lotsizesquarefeet              0
propertycountylandusecode      0
rawcensustractandblock         0
regionidcity                   0
regionidzip                    0
roomcnt                        0
year_built                     0
structuretaxvaluedollarcnt     0
tax_value                      0
assessmentyear                 0
landtaxvaluedollarcnt          0
tax_amount                     0
censustractandblock            0
logerror                       0
transactiondate                0
propertylandusedesc            0
LA                             0
Orange                         0
Ventura                        0
age       

In [10]:
train = train.dropna()
validate = validate.dropna()
test = test.dropna()

In [11]:
# target = 'logerror'
# X_train = train.drop(columns=[target])
# y_train = train[target]
    
#     # split validate into X (dataframe, drop target) & y (series, keep target only)
# X_validate = validate.drop(columns=[target])
# y_validate = validate[target]
    
#     # split test into X (dataframe, drop target) & y (series, keep target only)
# X_test = test.drop(columns=[target])
# y_test = test[target]

In [12]:
X_train, y_train, X_validate, y_validate, X_test, y_test = split_X_y(train, validate, test, 'logerror')

In [13]:
X_train

,propertylandusetypeid,parcel_id,id,baths,beds,calculatedbathnbr,sqft,finishedsquarefeet12,fullbathcnt,latitude,...,structure_dollar_sqft_bin,land_dollar_per_sqft,lot_dollar_sqft_bin,bath_bed_ratio,cola,county,los_angeles,orange,ventura,bathsandbeds
38772,261.0,11347306,505981,3.0,4.0,3.0,2846.0,2846.0,3.0,34462144.0,...,0.4,27.639819,0.3,0.750000,0,los_angeles,1,0,0,7.0
6386,261.0,13993394,1711476,1.0,5.0,1.0,1448.0,1448.0,1.0,33828474.0,...,0.1,18.834955,0.2,0.200000,0,orange,0,1,0,6.0
36392,261.0,14146719,2188838,2.0,4.0,2.0,2148.0,2148.0,2.0,33901265.0,...,0.1,3.582518,0.1,0.500000,0,orange,0,1,0,6.0
2872,261.0,14381521,1966277,2.5,4.0,2.5,2490.0,2490.0,2.0,33593821.0,...,0.2,61.319347,0.4,0.625000,0,orange,0,1,0,6.5
45193,261.0,10934407,598751,2.0,3.0,2.0,1446.0,1446.0,2.0,34123784.0,...,0.4,14.710199,0.2,0.666667,1,los_angeles,1,0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58249,261.0,12689886,2573422,3.0,4.0,3.0,3654.0,3654.0,3.0,33786759.0,...,0.4,74.984787,0.4,0.750000,0,los_angeles,1,0,0,7.0
10491,261.0,12756637,1783941,1.0,3.0,1.0,1044.0,1044.0,1.0,33901844.0,...,0.4,34.044137,0.3,0.333333,0,los_angeles,1,0,0,4.0
54469,261.0,13936053,2635062,2.0,3.0,2.0,1373.0,1373.0,2.0,33802487.0,...,0.2,44.700703,0.3,0.666667,0,orange,0,1,0,5.0
74787,261.0,12259922,1155128,1.0,3.0,1.0,962.0,962.0,1.0,33909932.0,...,0.1,17.140242,0.2,0.333333,0,los_angeles,1,0,0,4.0


In [14]:
def standard_scale_data(X_train, X_validate, X_test):
    """
    Takes in X_train, X_validate and X_test dfs with numeric values only
    Returns scaler, X_train_scaled, X_validate_scaled, X_test_scaled dfs
    """
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns = X_train.columns)
    X_validate_scaled = pd.DataFrame(scaler.transform(X_validate), index = X_validate.index, columns = X_validate.columns)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns = X_test.columns)
    return X_train_scaled, X_validate_scaled, X_test_scaled

In [15]:
def get_object_cols(df):
    '''
    This function takes in a dataframe and identifies the columns that are object types
    and returns a list of those column names. 
    '''
    # create a mask of columns whether they are object type or not
    mask = np.array(df.dtypes == "object")

        
    # get a list of the column names that are objects (from the mask)
    object_cols = df.iloc[:, mask].columns.tolist()
    
    return object_cols

In [16]:
def get_numeric_X_cols(X_train, object_cols):
    '''
    takes in a dataframe and list of object column names
    and returns a list of all other columns names, the non-objects. 
    '''
    numeric_cols = [col for col in X_train.columns.values if col not in object_cols]
    
    return numeric_cols

In [17]:
obj_cols = get_object_cols(X_train)

In [18]:
num_cols = get_numeric_X_cols(X_train, obj_cols)

In [19]:
X_train_scaled, X_validate_scaled, X_test_scaled = standard_scale_data(X_train[num_cols],X_validate[num_cols],X_test[num_cols])

In [20]:
X_train_scaled

,propertylandusetypeid,parcel_id,id,baths,beds,calculatedbathnbr,sqft,finishedsquarefeet12,fullbathcnt,latitude,...,structure_dollar_per_sqft,structure_dollar_sqft_bin,land_dollar_per_sqft,lot_dollar_sqft_bin,bath_bed_ratio,cola,los_angeles,orange,ventura,bathsandbeds
38772,-0.032276,-0.685718,-1.155051,0.996206,0.868302,0.996306,1.537375,1.538834,1.086055,1.594207,...,0.294510,0.905188,-0.284933,0.099635,0.305970,-0.527322,0.730536,-0.601758,-0.299127,1.038911
6386,-0.032276,0.452227,0.245436,-1.431686,2.013324,-1.431723,-0.473773,-0.477393,-1.383285,-0.721164,...,-1.002328,-1.208471,-0.416846,-0.682529,-2.163697,-0.527322,-1.368858,1.661797,-0.299127,0.380940
36392,-0.032276,0.518165,0.800013,-0.217740,0.868302,-0.217709,0.533240,0.532163,-0.148615,-0.455192,...,-0.968817,-1.208471,-0.645356,-1.464692,-0.816606,-0.527322,-1.368858,1.661797,-0.299127,0.380940
2872,-0.032276,0.619141,0.541452,0.389233,0.868302,0.389299,1.025237,1.025403,-0.148615,-1.578564,...,-0.381934,-0.503918,0.219649,0.881798,-0.255318,-0.527322,-1.368858,1.661797,-0.299127,0.709926
45193,-0.032276,-0.863285,-1.047275,-0.217740,-0.276720,-0.217709,-0.476651,-0.480278,-0.148615,0.357871,...,0.288102,0.905188,-0.478642,-0.682529,-0.068222,1.896373,0.730536,-0.601758,-0.299127,-0.277030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58249,-0.032276,-0.108344,1.246804,0.996206,0.868302,0.996306,2.699756,2.704150,1.086055,-0.873587,...,0.257163,0.905188,0.424382,0.881798,0.305970,-0.527322,0.730536,-0.601758,-0.299127,1.038911
10491,-0.032276,-0.079638,0.329623,-1.431686,-0.276720,-1.431723,-1.054964,-1.060051,-1.383285,-0.453077,...,0.816438,0.905188,-0.188984,0.099635,-1.564990,-0.527322,0.730536,-0.601758,-0.299127,-0.935001
54469,-0.032276,0.427568,1.318415,-0.217740,-0.276720,-0.217709,-0.581668,-0.585560,-0.148615,-0.816118,...,-0.234909,-0.503918,-0.029329,0.099635,-0.068222,-0.527322,-1.368858,1.661797,-0.299127,-0.277030
74787,-0.032276,-0.293249,-0.400903,-1.431686,-0.276720,-1.431723,-1.172928,-1.178313,-1.383285,-0.423524,...,-0.800836,-1.208471,-0.442236,-0.682529,-1.564990,-0.527322,0.730536,-0.601758,-0.299127,-0.935001


In [21]:
number_of_features_list=np.arange(1,40)
high_score=0

#Variable to store the optimum features
number_of_features=0           
score_list =[]

for n in range(len(number_of_features_list)):
    model = LinearRegression()
    rfe = RFE(model,number_of_features_list[n])
    X_train_rfe = rfe.fit_transform(X_train_scaled,y_train)
    X_test_rfe = rfe.transform(X_test_scaled)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        number_of_features = number_of_features_list[n]

print("Optimum number of features: %d" %number_of_features)
print("Score with %d features: %f" % (number_of_features, high_score))

Optimum number of features: 18
Score with 18 features: 0.003787


In [26]:
cols = list(X_train_scaled.columns)
model = LinearRegression()

#Initializing RFE model
rfe = RFE(model, 18)

#Transforming data using RFE
X_rfe = rfe.fit_transform(X_train_scaled,y_train)  

#Fitting the data to model
model.fit(X_rfe,y_train)
temp = pd.Series(rfe.support_,index = cols)
selected_features_rfe = temp[temp==True].index

print(selected_features_rfe)

Index(['baths', 'beds', 'calculatedbathnbr', 'rawcensustractandblock',
       'structuretaxvaluedollarcnt', 'tax_value', 'landtaxvaluedollarcnt',
       'tax_amount', 'censustractandblock', 'LA', 'Orange', 'Ventura',
       'structure_dollar_per_sqft', 'cola', 'los_angeles', 'orange', 'ventura',
       'bathsandbeds'],
      dtype='object')


In [23]:
from sklearn.metrics import mean_squared_error, r2_score
#turning my y_train and y_validate to dataframes so we can append new columns
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

#logerror mean
logerror_pred_mean = y_train['logerror'].mean()
y_train['logerror_pred_mean'] = logerror_pred_mean
y_validate['logerror_pred_mean'] = logerror_pred_mean

#logerror_median
logerror_pred_median = y_train['logerror'].median()
y_train['logerror_pred_median'] = logerror_pred_median
y_validate['logerror_pred_median'] = logerror_pred_median

#RMSE of logerror_pred_mean
rmse_train = mean_squared_error(y_train.logerror, y_train.logerror_pred_mean)**(1/2)
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_mean)**(1/2)

print('             BASELINE')
print('----------------------------------')
print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))
print('----------------------------------')

#RMSE of logerror_pred_median
rmse_train = mean_squared_error(y_train.logerror, y_train.logerror_pred_median)**(1/2)
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_median)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate, 2))
print('----------------------------------')
r2_baseline = r2_score(y_validate.logerror, y_validate.logerror_pred_mean)
print(f'The r^2 score for baseline is {r2_baseline}')

             BASELINE
----------------------------------
RMSE using Mean
Train/In-Sample:  0.17 
Validate/Out-of-Sample:  0.16
----------------------------------
RMSE using Median
Train/In-Sample:  0.17 
Validate/Out-of-Sample:  0.16
----------------------------------
The r^2 score for baseline is -2.171622815549945e-05


In [27]:
selected_features = ['baths', 'beds', 'calculatedbathnbr', 'rawcensustractandblock',
       'structuretaxvaluedollarcnt', 'tax_value', 'landtaxvaluedollarcnt',
       'tax_amount', 'censustractandblock', 'LA', 'Orange', 'Ventura',
       'structure_dollar_per_sqft', 'cola', 'los_angeles', 'orange', 'ventura',
       'bathsandbeds']
#Our first model: LinearRegression
#create model object
lm = LinearRegression(normalize=True)

#fit the model to our training data, specify column since it is a dataframe
lm.fit(X_train_scaled[selected_features],y_train.logerror)

#predict train
y_train['logerror_pred_lm'] = lm.predict(X_train_scaled[selected_features])

#evaluate the RMSE for train
rmse_train = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm)**(1/2)

#predict validate
y_validate['logerror_pred_lm'] = lm.predict(X_validate_scaled[selected_features])

#evaluate the RMSE for validate
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm)**(1/2)

print("RMSE for OLS using LinearRegression:\nTraining/In-Sample: ",rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)
r2_score_lm = r2_score(y_validate.logerror, y_validate.logerror_pred_lm)
print(f'The r^2 value for linear regression is {round(r2_score_lm,3)}')

RMSE for OLS using LinearRegression:
Training/In-Sample:  0.16836398629197516 
Validation/Out-of-Sample:  0.2710654621749944
The r^2 value for linear regression is -1.758


In [28]:
selected_features = ['baths', 'beds', 'calculatedbathnbr', 'rawcensustractandblock',
       'structuretaxvaluedollarcnt', 'tax_value', 'landtaxvaluedollarcnt',
       'tax_amount', 'censustractandblock', 'LA', 'Orange', 'Ventura',
       'structure_dollar_per_sqft', 'cola', 'los_angeles', 'orange', 'ventura',
       'bathsandbeds']
#LassoLars
#create model object
lars = LassoLars(alpha=1.0)

#fit the model to our training data, specify column since it is a dataframe
lars.fit(X_train_scaled[selected_features],y_train.logerror)

#predict train
y_train['logerror_pred_lars'] = lars.predict(X_train_scaled[selected_features])

#evaluate the RMSE for train
rmse_train = mean_squared_error(y_train.logerror, y_train.logerror_pred_lars)**(1/2)

#predict validate
y_validate['logerror_pred_lars'] = lars.predict(X_validate_scaled[selected_features])

#evaluate the RMSE for validate
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lars)**(1/2)

print("RMSE for OLS using LassoLars:\nTraining/In-Sample: ",rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)
r2_score_lars = r2_score(y_validate.logerror, y_validate.logerror_pred_lars)
print(f'The r^2 value for LassoLars regression is {round(r2_score_lars,3)}')

RMSE for OLS using LassoLars:
Training/In-Sample:  0.16880204139702645 
Validation/Out-of-Sample:  0.16321483314718824
The r^2 value for LassoLars regression is -0.0


In [29]:
#TweedieRegressor
#create model object
glm = TweedieRegressor(power = 0,alpha=1.0)

#fit the model to our training data, specify column since it is a dataframe
glm.fit(X_train_scaled[selected_features],y_train.logerror)

#predict train
y_train['logerror_pred_glm'] = glm.predict(X_train_scaled[selected_features])

#evaluate the RMSE for train
rmse_train = mean_squared_error(y_train.logerror, y_train.logerror_pred_glm)**(1/2)

#predict validate
y_validate['logerror_pred_glm'] = glm.predict(X_validate_scaled[selected_features])

#evaluate the RMSE for validate
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_glm)**(1/2)

print("RMSE for OLS using TweedieRegressor:\nTraining/In-Sample: ",rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)
r2_score_glm = r2_score(y_validate.logerror, y_validate.logerror_pred_glm)
print(f'The r^2 value for TweedieRegressor regression is {round(r2_score_glm,3)}')

RMSE for OLS using TweedieRegressor:
Training/In-Sample:  0.16853554236818513 
Validation/Out-of-Sample:  0.16298701693043174
The r^2 value for TweedieRegressor regression is 0.003


In [32]:
from sklearn.preprocessing import PolynomialFeatures

#making polynomial features to get a new set of features, I found that degree=3 yielded the best results.
pf = PolynomialFeatures(degree=4)

#fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train_scaled[selected_features])

#transform X_validate_scaled, and X_test_scaled
X_validate_degree2 = pf.transform(X_validate_scaled[selected_features])
X_test_degree2 = pf.transform(X_test_scaled[selected_features])

#linear regresson with 2 degrees
lm2 = LinearRegression(normalize=True)

#fit the model to our training data, specify column since it is a dataframe
lm2.fit(X_train_scaled[selected_features],y_train.logerror)

#predict train
y_train['logerror_pred_lm2'] = lm2.predict(X_train_scaled[selected_features])

#evaluate the RMSE for train
rmse_train = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm2)**(1/2)

#predict validate
y_validate['logerror_pred_lm2'] = lm2.predict(X_validate_scaled[selected_features])

#evaluate the RMSE for validate
rmse_validate = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm2)**(1/2)

print("RMSE for OLS using Polynomial Regression (2 degrees):\nTraining/In-Sample: ",rmse_train, 
      "\nValidation/Out-of-Sample: ", rmse_validate)
r2_score_lm2 = r2_score(y_validate.logerror, y_validate.logerror_pred_lm2)
print(f'The r^2 value for Polynomial Regression (2 degrees) is {round(r2_score_lm2,3)}')

RMSE for OLS using Polynomial Regression (2 degrees):
Training/In-Sample:  0.16836398629197516 
Validation/Out-of-Sample:  0.2710654621749944
The r^2 value for Polynomial Regression (2 degrees) is -1.758
